<a href="https://colab.research.google.com/github/shravya004/Ai-Weed-Sprayer/blob/main/FINAL_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- 1. Install Libraries ---
print("🚀 Installing necessary libraries...")
!pip install -q kaggle qiskit qiskit-aer qiskit-machine-learning

# --- 2. Configure Kaggle API ---
print("\n🔑 Please upload your kaggle.json file")
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
print("\n✅ Kaggle API configured.")

# --- 3. Download and Unzip Dataset ---
print("\nDownloading and unzipping the PlantVillage dataset...")
!kaggle datasets download -d emmarex/plantdisease
!unzip -q plantdisease.zip
print("\n✅ Dataset is ready.")

🚀 Installing necessary libraries...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 138.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 145.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 18.1 MB/s eta 0:00:00

🔑 Please upload your kaggle.json file


Saving kaggle.json to kaggle.json

✅ Kaggle API configured.

Dataset URL: https://www.kaggle.com/datasets/emmarex/plantdisease
License(s): unknown
 97% 641M/658M [00:01<00:00, 280MB/s]
100% 658M/658M [00:01<00:00, 381MB/s]

✅ Dataset is ready.


In [9]:
!pip install qiskit_algorithms

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 7.9 MB/s eta 0:00:00


In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

IMAGE_SIZE = 32
DATA_DIR = 'PlantVillage/'

print("💾 Loading all images from the dataset...")
data = []
labels = []
class_names = sorted(os.listdir(DATA_DIR))
num_classes = len(class_names)
class_to_int = {name: i for i, name in enumerate(class_names)}

for class_name in class_names:
    class_path = os.path.join(DATA_DIR, class_name)
    class_label = class_to_int[class_name]
    for img_name in os.listdir(class_path):
        try:
            img_path = os.path.join(class_path, img_name)
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            resized_image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
            data.append(resized_image)
            labels.append(class_label)
        except Exception:
            pass

X_full = np.array(data)
y_full = np.array(labels)

print(f"\n✅ Data loading complete. Found {len(X_full)} images across {num_classes} classes.")

💾 Loading all images from the dataset...

✅ Data loading complete. Found 20638 images across 15 classes.


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# --- Build and Train Multi-Class CNN ---
# This is the corrected model definition
print("\n🧠 Building the multi-class CNN model...")
cnn_model_multi = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'), # Increased dense layer size
    Dropout(0.5),
    Dense(num_classes, activation='softmax') # num_classes is defined in Cell 2
])

cnn_model_multi.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn_model_multi.summary()

# --- Use Data Augmentation (Improvement 1) ---
print("\nApplying data augmentation...")
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train_cnn_full)

# --- Train the model using the generator ---
print("\n🏋️ Training the multi-class CNN model with Data Augmentation...")
history = cnn_model_multi.fit(
    datagen.flow(X_train_cnn_full, y_train_full, batch_size=32),
    epochs=40, # Train for more epochs as the data is more varied
    validation_data=(X_test_cnn_full, y_test_full),
    steps_per_epoch=len(X_train_cnn_full) // 32, # Number of batches per epoch
    verbose=1
)

# --- Evaluate ---
loss, multi_class_cnn_accuracy = cnn_model_multi.evaluate(X_test_cnn_full, y_test_full, verbose=0)
print(f"\n✅ Final Multi-Class CNN Test Accuracy: {multi_class_cnn_accuracy:.4f}")


🧠 Building the multi-class CNN model...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │         1,935 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 315,791 (1.20 MB)

...


Applying data augmentation...

🏋️ Training the multi-class CNN model with Data Augmentation...
Epoch 1/40


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


515/515 ━━━━━━━━━━━━━━━━━━━━ 15s 20ms/step - accuracy: 0.2009 - loss: 2.4141 - val_accuracy: 0.3832 - val_loss: 2.0024
Epoch 2/40
  1/515 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.3125 - loss: 2.0361

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


515/515 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3125 - loss: 2.0361 - val_accuracy: 0.3726 - val_loss: 2.0222
Epoch 3/40
515/515 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.3384 - loss: 2.0297 - val_accuracy: 0.3336 - val_loss: 2.2078
Epoch 4/40
515/515 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4062 - loss: 1.8529 - val_accuracy: 0.3193 - val_loss: 2.2873
Epoch 5/40
515/515 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - accuracy: 0.3996 - loss: 1.8389 - val_accuracy: 0.3481 - val_loss: 2.4095
Epoch 6/40
515/515 ━━━━━━━━━━━━━━━━━━━━ 0s 576us/step - accuracy: 0.2812 - loss: 1.9127 - val_accuracy: 0.3515 - val_loss: 2.3584
Epoch 7/40
515/515 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - accuracy: 0.4323 - loss: 1.7304 - val_accuracy: 0.4138 - val_loss: 2.0514
Epoch 8/40
515/515 ━━━━━━━━━━━━━━━━━━━━ 0s 568us/step - accuracy: 0.5000 - loss: 1.6759 - val_accuracy: 0.4196 - val_loss: 2.0103
Epoch 9/40
515/515 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.4579 - loss: 1.6581 - val_accuracy: 0.393

In [5]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# --- Experiment 2: Binary Classification Setup ---
print("--- Experiment 2: Binary Classification Setup ---")

# --- 1. Create the Binary Subset ---
CLASS_HEALTHY = 'Pepper__bell___healthy'
CLASS_DISEASED = 'Pepper__bell___Bacterial_spot'
SELECTED_CLASSES = [CLASS_HEALTHY, CLASS_DISEASED] # For classification reports later

print(f"Filtering for 2 classes:\n1. {CLASS_HEALTHY}\n2. {CLASS_DISEASED}")

# Get the integer labels for these classes from the map we created in Cell 2
healthy_label = class_to_int[CLASS_HEALTHY]
diseased_label = class_to_int[CLASS_DISEASED]

# Find the indices in the full dataset
healthy_indices = np.where(y_full == healthy_label)[0]
diseased_indices = np.where(y_full == diseased_label)[0]

# Combine these indices
binary_indices = np.concatenate([healthy_indices, diseased_indices])

# Filter the full data (X_full and y_full are from Cell 2)
X_binary = X_full[binary_indices]
y_binary_original_labels = y_full[binary_indices]

# Convert labels to 0 (healthy) and 1 (diseased)
y_binary = np.where(y_binary_original_labels == healthy_label, 0, 1)

# Shuffle them together
X_binary, y_binary = shuffle(X_binary, y_binary, random_state=42)
print(f"Created binary dataset with {len(X_binary)} samples.")

# --- 2. Split and Preprocess Binary Data ---
# We create a new set of train/test variables for this binary experiment
X_train, X_test, y_train, y_test = train_test_split(X_binary, y_binary, test_size=0.2, random_state=42)

# Reshape for scaler (N_samples, N_features)
X_train_flat = X_train.reshape(-1, IMAGE_SIZE * IMAGE_SIZE)
X_test_flat = X_test.reshape(-1, IMAGE_SIZE * IMAGE_SIZE)

# Create a *new* scaler fit *only* on the binary training data
scaler_binary = StandardScaler().fit(X_train_flat)
X_train_scaled = scaler_binary.transform(X_train_flat)
X_test_scaled = scaler_binary.transform(X_test_flat)

print("Binary data split and scaled.")

# --- 3. PCA for SVM/QSVM ---
# This data will be used for the SVM and QSVM models
N_COMPONENTS = 8 # Using 8 as you had in your notebook
print(f"Applying PCA, reducing to {N_COMPONENTS} components...")

pca = PCA(n_components=N_COMPONENTS)
# Fit PCA *only* on the training data
X_train_pca = pca.fit_transform(X_train_scaled)
# Transform test data with the *same* fitted PCA
X_test_pca = pca.transform(X_test_scaled)

print("✅ Binary data prepared and PCA applied.")
print(f"Shape of X_train_pca (for SVM/QSVM): {X_train_pca.shape}")
print(f"Shape of X_test_pca (for SVM/QSVM): {X_test_pca.shape}")

--- Experiment 2: Binary Classification Setup ---
Filtering for 2 classes:
1. Pepper__bell___healthy
2. Pepper__bell___Bacterial_spot
Created binary dataset with 2475 samples.
Binary data split and scaled.
Applying PCA, reducing to 8 components...
✅ Binary data prepared and PCA applied.
Shape of X_train_pca (for SVM/QSVM): (1980, 8)
Shape of X_test_pca (for SVM/QSVM): (495, 8)


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms import QSVC

# --- 1. Binary CNN ---
print("🧠 Training Binary CNN model...")
# Reshape the scaled data (not PCA'd) for the CNN
X_train_cnn_binary = X_train_scaled.reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)
X_test_cnn_binary = X_test_scaled.reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)

binary_cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid') # Binary (1 neuron + sigmoid)
])

binary_cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the binary CNN
binary_cnn_model.fit(
    X_train_cnn_binary,
    y_train,
    epochs=20,
    validation_split=0.2,
    verbose=0 # verbose=0 to keep the output clean
)

# Evaluate the binary CNN
_, binary_cnn_accuracy = binary_cnn_model.evaluate(X_test_cnn_binary, y_test, verbose=0)
print(f"✅ Binary CNN Test Accuracy: {binary_cnn_accuracy:.4f}\n")


# --- 2. Optimized Classical SVM (Improvement 2) ---
print("🤖 Optimizing and Training Classical SVM...")
# Define a grid of parameters to search
param_grid = {
    'C': [0.1, 1, 10, 100],          # Regularization parameter
    'gamma': [1, 0.1, 0.01, 0.001], # Kernel coefficient
    'kernel': ['rbf']
}

# Create a GridSearchCV object (cv=3 means 3-fold cross-validation)
grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=0, cv=3)

# Fit the grid search to the binary PCA data (X_train_pca, y_train)
grid.fit(X_train_pca, y_train)

# Get the best model
best_svc = grid.best_estimator_
svc_score = best_svc.score(X_test_pca, y_test) # Score on the test set

print(f"Best SVM parameters found: {grid.best_params_}")
print(f"✅ Optimized Classical SVM Accuracy: {svc_score:.4f}\n")


# --- 3. Quantum SVM (QSVM) ---
print("⚛️  Training QSVM...")
# Note: QSVM is slow, so we train on a smaller subset of the PCA'd training data
TRAIN_SUBSET_SIZE = 150
X_train_q = X_train_pca[:TRAIN_SUBSET_SIZE]
y_train_q = y_train[:TRAIN_SUBSET_SIZE]

print(f"Using a subset of {TRAIN_SUBSET_SIZE} samples for QSVM training...")

# Define the quantum feature map (N_COMPONENTS was defined in Cell 4)
feature_map = ZZFeatureMap(feature_dimension=N_COMPONENTS, reps=2, entanglement='linear')

# Create the quantum kernel
quantum_kernel = FidelityQuantumKernel(feature_map=feature_map)

# Create and configure the QSVC
qsvc = QSVC(quantum_kernel=quantum_kernel)

# Fit the QSVM
qsvc.fit(X_train_q, y_train_q)

# Evaluate the QSVM on the full PCA'd test set
qsvc_score = qsvc.score(X_test_pca, y_test)
print(f"✅ QSVM Accuracy: {qsvc_score:.4f}\n")

🧠 Training Binary CNN model...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


✅ Binary CNN Test Accuracy: 0.8869

🤖 Optimizing and Training Classical SVM...
Best SVM parameters found: {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
✅ Optimized Classical SVM Accuracy: 0.8869

⚛️  Training QSVM...
Using a subset of 150 samples for QSVM training...
✅ QSVM Accuracy: 0.5939



In [10]:
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit_algorithms.optimizers import COBYLA
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit_machine_learning.circuit.library import QNNCircuit

print("⚛️ Training a Variational Quantum Classifier (VQC)...")
# N_COMPONENTS, X_train_pca, y_train, X_test_pca, y_test are from Cell 4

# 1. We still need a feature map to encode the data
vqc_feature_map = ZZFeatureMap(feature_dimension=N_COMPONENTS, reps=2, entanglement='linear')

# 2. We need an "ansatz" - this is the trainable part of the quantum circuit
vqc_ansatz = RealAmplitudes(num_qubits=N_COMPONENTS, reps=3)

# 3. Create the VQC
vqc = VQC(
    feature_map=vqc_feature_map,
    ansatz=vqc_ansatz,
    optimizer=COBYLA(maxiter=100), # 100 iterations
)

# Train on the same subset as the QSVM
print(f"Using a subset of {TRAIN_SUBSET_SIZE} samples for VQC training...")
vqc.fit(X_train_q, y_train_q) # X_train_q, y_train_q are from Cell 5

# Evaluate the VQC
vqc_score = vqc.score(X_test_pca, y_test)
print(f"\n✅ Variational Quantum Classifier (VQC) Accuracy: {vqc_score:.4f}")

⚛️ Training a Variational Quantum Classifier (VQC)...
Using a subset of 150 samples for VQC training...

✅ Variational Quantum Classifier (VQC) Accuracy: 0.4949


In [7]:
import pandas as pd

# --- Cell 6: Final Tabulated Results ---

print("--- Final Model Performance (Fair Comparison on 2-Class Task) ---")
print("This table compares all three models on the same binary problem.\n")

try:
    # --- 1. Create the Apples-to-Apples Comparison Table ---
    fair_comparison_data = {
        'Model': [
            'Binary CNN',
            'Optimized Classical SVM',
            'QSVM'
        ],
        'Task': [
            '2 Classes (Healthy vs. Diseased)',
            '2 Classes (Healthy vs. Diseased)',
            '2 Classes (Healthy vs. Diseased)'
        ],
        'Test Accuracy': [
            binary_cnn_accuracy, # From Cell 5
            svc_score,           # From Cell 5
            qsvc_score           # From Cell 5
        ]
    }

    df_fair = pd.DataFrame(fair_comparison_data)

    # Sort by accuracy to easily see the best performer
    df_fair = df_fair.sort_values(by='Test Accuracy', ascending=False)

    # Format accuracy as a percentage for readability
    df_fair['Test Accuracy'] = df_fair['Test Accuracy'].apply(lambda x: f"{x:.2%}")

    # Print the clean table without the index column
    print(df_fair.to_string(index=False))

except NameError as e:
    print(f"Error: Could not create the fair comparison table. A variable is missing: {e}")
    print("Please make sure you have successfully run Cell 5.")

# --- 2. Create the Overall Summary Table ---
print("\n\n--- Overall Summary of All Experiments ---")
print("This table includes the multi-class CNN for context.\n")

try:
    overall_data = {
        'Model': [
            'Multi-Class CNN',
            'Binary CNN',
            'Optimized Classical SVM',
            'QSVM'
        ],
        'Task': [
            f'{num_classes} Classes (Full Dataset)', # num_classes from Cell 2
            '2 Classes (Healthy vs. Diseased)',
            '2 Classes (Healthy vs. Diseased)',
            '2 Classes (Healthy vs. Diseased)'
        ],
        'Test Accuracy': [
            multi_class_cnn_accuracy, # From Cell 3
            binary_cnn_accuracy,
            svc_score,
            qsvc_score
        ]
    }

    df_overall = pd.DataFrame(overall_data)
    df_overall['Test Accuracy'] = df_overall['Test Accuracy'].apply(lambda x: f"{x:.2%}")
    print(df_overall.to_string(index=False))

except NameError as e:
    print(f"Error: Could not create the overall summary. A variable is missing: {e}")
    print("Please make sure you have successfully run Cells 3 and 5.")

--- Final Model Performance (Fair Comparison on 2-Class Task) ---
This table compares all three models on the same binary problem.

                  Model                             Task Test Accuracy
             Binary CNN 2 Classes (Healthy vs. Diseased)        88.69%
Optimized Classical SVM 2 Classes (Healthy vs. Diseased)        88.69%
                   QSVM 2 Classes (Healthy vs. Diseased)        59.39%


--- Overall Summary of All Experiments ---
This table includes the multi-class CNN for context.

                  Model                             Task Test Accuracy
        Multi-Class CNN        15 Classes (Full Dataset)        45.42%
             Binary CNN 2 Classes (Healthy vs. Diseased)        88.69%
Optimized Classical SVM 2 Classes (Healthy vs. Diseased)        88.69%
                   QSVM 2 Classes (Healthy vs. Diseased)        59.39%
